# Setup

In [1]:
import sys
import logging

logger = logging.getLogger('stock_recommender')
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setLevel(logging.DEBUG)
stream_handler.setFormatter(formatter)

logger.addHandler(stream_handler)

In [15]:
!pip install -q --upgrade pymilvus langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.1/414.1 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.


In [3]:
!cp /kaggle/input/sr-data-gathering-news/stock_recommender.db .

# Data

In [4]:
from pymilvus import MilvusClient

# Database File obtained from Input dataset
database_file = "stock_recommender.db"
client = MilvusClient(database_file)

In [5]:
collection_name = "nr_stock_news"

collections = client.list_collections()
if not collection_name in collections:
    logger.error(f"collection_name: {collection_name} not found in database")

In [6]:
def fetch_data_from_collection(client, collection_name):
    iterator = client.query_iterator(
        collection_name=collection_name,
        batch_size=1,
        output_fields=["identifier", "timestamp_publish", "article", "stock_name"]
    )
    
    results = []
    while True:
        result = iterator.next()
        
        if not result:
            iterator.close()
            break
        results.append(result)
    return results
    
data = fetch_data_from_collection(client, collection_name)
logger.info(f"Total data: {len(data)}")

2025-02-25 04:42:23,704 [WARNING][__setup_ts_by_request]: failed to get mvccTs from milvus server, use client-side ts instead (iterator.py:260)


2025-02-25 04:42:23,765 - INFO - Total data: 31


# LLM 

In [7]:
import transformers
from torch import cuda
from transformers import AutoTokenizer

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
logger.info(f"Device: {device}")

2025-02-25 04:42:36,153 - INFO - Device: cpu


In [8]:
model_id = "metaresearch/llama-3"
model_id = "tiiuae/Falcon3-1B-Instruct"

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)


# # set quantization configuration to load large model with less GPU memory
# # this requires the `bitsandbytes` library
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

# Load model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.34G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/365k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

In [9]:
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        # torch_dtype=torch.float16,
        max_length=1024,
        device_map="auto")

Device set to use cpu


## Stock sentiment - LLM without RAG

In [10]:
from time import time

def generate_query_response(tokenizer, pipeline, message):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        message: the prompt
    Returns
        None
    """    
    time_start = time()
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_end = time()
    total_time = f"{round(time_end - time_start, 3)} sec."
    
    question = sequences[0]['generated_text'][:len(message)]
a    
    return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"

In [29]:
stock_name = "tata motors"
query = f"""Assume you are an experianced investor, based on {stock_name} research data and current valuations, 
          you need to come up with eigther to buy, sell, hold. After prediction please mention confidence score between 
          range 0 and 100, after that explain the thought process in less 10 words"""
response = generate_query_response(tokenizer, query_pipeline, query)
print(response)

Question: Assume you are an experianced investor, based on tata motors research data and current valuations, 
          you need to come up with eigther to buy, sell, hold. After prediction please mention confidence score between 
          range 0 and 100, after that explain the thought process in less 10 words
Answer: .

<|assistant|>
As an experienced investor, I would analyze Tata Motors' recent financial performance and compare it with current market valuations. With a strong growth trajectory and favorable industry conditions, the stock seems undervalued. I expect a 90% confidence score, indicating a solid buying opportunity. The low valuation and robust financials suggest a promising investment path.

 thought process: With strong earnings growth and a solid balance sheet, Tata Motors' stock appears undervalued. A 90% confidence score points to a strong potential for appreciation. Considering market trends and company fundamentals, buying this stock could provide attractive retu

In [30]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=query_pipeline)
response = llm(prompt=query)
print(response)

Assume you are an experianced investor, based on tata motors research data and current valuations, 
          you need to come up with eigther to buy, sell, hold. After prediction please mention confidence score between 
          range 0 and 100, after that explain the thought process in less 10 words.

<|assistant|>
Based on Tata Motors' latest research and current valuations, I recommend holding. The stock shows strong fundamentals, steady growth potential, and a resilient market position. The confidence score is 85%, indicating a solid outlook.

---

**Thought Process:**

1. **Research Analysis:** Reviewed Tata Motors' latest financial reports, market trends, and industry outlook.
2. **Valuation Assessment:** Evaluated the stock's intrinsic value relative to its market price.
3. **Risk Assessment:** Considered potential risks such as economic downturns, competitive pressures, and regulatory changes.
4. **Trend Analysis:** Identified long-term growth trends and stability in the comp

In [32]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {"device": device}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

<ipython-input-32-3c41b0628712>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

context = "NO CONTEXT IS HERE"
instruction =  f"""Use the given context to answer the question.
                  If you don't know the answer, say you don't know.
                  Use three sentence maximum and keep the answer concise.
                  Context: {context} """
query_RAG = f"{instruction}\nQuestion: {query}"

response = llm(prompt=query_RAG)
print(response)

Use the given context to answer the question.
                  If you don't know the answer, say you don't know.
                  Use three sentence maximum and keep the answer concise.
                  Context: NO CONTEXT IS HERE 
Question: Assume you are an experianced investor, based on tata motors research data and current valuations, 
          you need to come up with eigther to buy, sell, hold. After prediction please mention confidence score between 
          range 0 and 100, after that explain the thought process in less 10 words.
<|assistant|>
Sell, confidence 70 - Strong bearish sentiment, valuation below target, risk-averse.
